In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

In [25]:
def estimate_ci_bootstrapping(negative, positive, num_iterations=1000,z=1.96):
    data = np.concatenate([np.ones(negative),np.zeros(positive)])
    bootstrap_samples = np.random.choice(a=data, size=(num_iterations, len(data)), replace=True)
    negative_rate = np.mean(bootstrap_samples, axis=1)
    mean_of_negative_rate = np.mean(negative_rate)
    sem_of_negative_rate = stats.sem(negative_rate)
    return mean_of_negative_rate, mean_of_negative_rate-1.96*sem_of_negative_rate, mean_of_negative_rate+1.96*sem_of_negative_rate

In [2]:
data_path = "../../data/NationalFunding/Data/RobustCheck/check_09_and_18/funding_year_check_result.xlsx"

In [18]:
data = pd.read_excel(data_path)
data.head()

,ost_bk,ut,year,title,label
0,65041047,WOS:000265236000001,2009,Symbolic Computations and Exact and Explicit S...,0.0
1,32590270,WOS:000266300300033,2009,Experiencing the pediatric intensive care unit...,0.0
2,32590344,WOS:000270759800006,2009,Determination of plutonium isotopes in waters ...,0.0
3,32590368,WOS:000265889000017,2009,Extremity Problems in Ostrich Chicks and Their...,0.0
4,32590526,WOS:000261739200016,2009,Neuroanatomy with a behavioral twist,0.0


In [20]:
data = data.replace(to_replace={'label':{0:'positive',1:'negative'}})
data = data[data['label'].isin(['negative','positive'])].groupby(['year','label']).size().reset_index(name='count')
data = data.pivot(index='year',columns='label',values='count').reset_index()
data.head()

label,year,negative,positive
0,2009,24,126
1,2018,8,142


In [26]:
res_df = []
for index, row in data.iterrows():
    year=row['year']
    negative_value = row['negative']
    positive_value = row['positive']
    mean, lower, upper = estimate_ci_bootstrapping(negative_value, positive_value)
    res_df.append(['global',year,mean,lower,upper])
res_df = pd.DataFrame(res_df, columns=['cntry','year','mean','lower','upper'])

In [27]:
res_df.to_csv("../../data/NationalFunding/Data/RobustCheck/check_09_and_18/ci_global.csv",index=False)